In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pdf2image import convert_from_path
poppler_path =".\\poppler\\Library\\bin"
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import pytesseract
pytesseract.pytesseract.tesseract_cmd = (
        r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    )
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
from langchain_openai import ChatOpenAI
import os

import pandas as pd

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embedding=OpenAIEmbeddings(model="text-embedding-3-small")

#file_name="8439-0"
#pdf_file = ".\\PDFs\\"+file_name+".pdf"
#pages = convert_from_path(pdf_file,poppler_path =poppler_path)


In [22]:
from langchain_core.pydantic_v1 import BaseModel, Field
class GPA_info(BaseModel):
    Name:str=Field(description='name of the college or University')
    Address:str=Field(description='address of the college or University')
    Year:str=Field(description="Year of Common Data Set, format should be same as below: '2023-2024'")
    GPA400:str=Field(description='Percent who had GPA of 4.0')
    GPA375:str=Field(description='Percent who had GPA between 3.75 and 3.99')
    GPA350:str=Field(description='Percent who had GPA between 3.50 and 3.74')
    GPA325:str=Field(description='Percent who had GPA between 3.25 and 3.49')
    GPA300:str=Field(description='Percent who had GPA between 3.00 and 3.24')
    GPA250:str=Field(description='Percent who had GPA between 2.50 and 2.99')
    GPA200:str=Field(description='Percent who had GPA between 2.0 and 2.49')
    GPA100:str=Field(description='Percent who had GPA between 1.0 and 1.99')
    GPA000:str=Field(description='Percent who had GPA below 1.0')


GPA_info_structured_output=llm.with_structured_output(GPA_info)
prompt = ChatPromptTemplate.from_messages([
    ("system", "Only based on documents content below, ouput information strictly based on format, do not make up numbers, if you cannot find information in the documents, output all zeros"),
    ("human", "documents：{documents}"),
])
GPA_chain=prompt | GPA_info_structured_output

In [23]:
df_check=pd.read_csv("GPAs.csv")

In [ ]:
path="PDFs"
items=os.scandir(path)
count=0
df=pd.DataFrame(columns=['postid','Name','ExtractName','Address','Year','GPA400','GPA375','GPA350','GPA325','GPA300','GPA250','GPA200','GPA100','GPA000'])
colleges_df=pd.read_csv("..//College-Information-LLM//KnowledgeBase//colleges.csv")
for item in items:
    if item.is_dir() and not(int(item.name) in df_check['postid'].unique()):
        filePath=os.path.join(path,item.name)
        files=os.scandir(filePath)
        for file in files:
            if file.is_file():
                pages=convert_from_path(file.path,poppler_path =poppler_path)
                if len(pages)<=100:
                    os.makedirs(file.path[:-4],exist_ok=True)
                    for index,page in enumerate(pages):
                        page.save(os.path.join(file.path[:-4],str(index))+'.jpg')
                        text = str(pytesseract.image_to_string(page))
                        if index==0:
                            faiss_index = FAISS.from_texts([text], embedding)
                        else:
                            faiss_index.add_texts([text])
                    docs_name=faiss_index.similarity_search("address information", k=2)
                    docs_GPA = faiss_index.similarity_search("GPA Score Range", k=2)
                    docs=docs_GPA+docs_name
                    response=GPA_chain.invoke({"documents":docs})
                    df2=pd.DataFrame([[
                        item.name,
                        colleges_df[colleges_df['postid']==int(item.name)]['name'].values[0],
                        response.Name,
                        response.Address,
                        response.Year,
                        response.GPA400,
                        response.GPA375,
                        response.GPA350,
                        response.GPA325,
                        response.GPA300,
                        response.GPA250,
                        response.GPA200,
                        response.GPA100,
                        response.GPA000
                    ]],columns=['postid','Name','ExtractName','Address','Year','GPA400','GPA375','GPA350','GPA325','GPA300','GPA250','GPA200','GPA100','GPA000'])
                    df=pd.concat([df,df2]).reset_index(drop=True)
                    count=count+1
                    df.to_csv("GPAs.csv")
        files.close()
items.close()          